# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
#%pip install langchain

In [ ]:
#%pip install langchain-openai

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#%pip install pandas

In [44]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [45]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [46]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [52]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [53]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [54]:
#chain = LLMChain(llm=llm, prompt=prompt)
chain = prompt | llm

In [55]:
product = "Rubber Ducks"
chain.invoke(product)

TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.

## SimpleSequentialChain

In [15]:
from langchain.chains import SimpleSequentialChain

In [16]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [17]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [19]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing the ultimate bath time companions - Rubber Ducks! These quacky little guys are waterproof, float effortlessly, and are guaranteed to make bath time more fun than a barrel of monkeys (and definitely less messy). So ditch the boring ol' bar of soap and upgrade to a Rubber Duck today. Your inner child will thank you!
I would rate the quality of Rubber Ducks as a bath time companion a solid 8 out of 10. They are fun, durable, and a great addition to any bath time routine.

> Finished chain.


{'input': 'Rubber Ducks',
 'output': 'I would rate the quality of Rubber Ducks as a bath time companion a solid 8 out of 10. They are fun, durable, and a great addition to any bath time routine.'}

**Repeat the above twice for different products**

## SequentialChain

In [20]:
from langchain.chains import SequentialChain

In [22]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )


In [23]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )


In [24]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [25]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [26]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [27]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Moyen goût, mousse faible, possible contrefaçon.',
 'en_short_review': 'Medium taste, weak foam, possible counterfeit.',
 'rating': 'I would rate the translation a 9 out of 10. The translation effectively captures the meaning and maintains the overall tone of the original phrase. The only minor improvement could be using "low foam" instead of "weak foam" to better convey the intended message.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [28]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [29]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [30]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [31]:
llm = ChatOpenAI(temperature=0)

In [32]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [35]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)

chain = (
    RunnablePassthrough()
    | {
        "input": RunnablePassthrough(),
        "chosen_prompt": router_chain,
    }
    | RunnableLambda(lambda x: prompt_templates[x["chosen_prompt"]].format(input=x["input"]))
    | llm
)



In [37]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")


TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [38]:
response

NameError: name 'response' is not defined

In [40]:
chain.invoke("What is black body radiation?")

TypeError: Expected mapping type as input to PromptTemplate. Received <class 'str'>.

In [89]:
chain.invoke("what is 2 + 2")

{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**